In [68]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sci
import pandas as pd

df_rep = pd.DataFrame({
    "max_voltage": [1.2, 3, 6],
    "val": [90.8, 94.9, 77.7],
    "max_current": [1.5] * 3,
    "E": [1, 2.7, 4.5],
})

R_err = [1e-3, 1e-3, 5e-3, 1e-2, 2e-2, 0, 5e-2]
int_RV = [1199.9, 3000.0, 6002.8]
int_RV_err = [0.0] * 3

for i in range(len(int_RV)):
    res = 0
    string = f"{int_RV[i]:.1f}"
    n =  len(string)
    for j in range(len(string)):
        if string[j].isdigit():
            res = res * 10 + int(string[j]) * R_err[-n+j]
    int_RV_err[i] = res / 10

print(int_RV)
print(int_RV_err)

[1199.9, 3000.0, 6002.8]
[2.625, 3.0, 6.08]


In [69]:
df_inter_conn = pd.DataFrame({
    "Rx0": [100.0, 500.0, 1000.0, 5000.0],
    "internal_RV": [0, 1199.9, 0, 3000.0],
    "U": [1.105, 1.032, 1.091, 2.9600],
    "internal_RA": [6.3, 29.4, 36.0, 36.0],
    "max_voltage": [1.2, 1.2, 1.2, 3],
    "max_current": [15, 3, 1.5, 1.5],
    "I": [10.08, 2.120, 1.042, 0.583],
})


Rx_int = df_inter_conn["U"] / df_inter_conn["I"] * 1000
Rx_intfixed = Rx_int - df_inter_conn["internal_RA"]

print(Rx_int, "\n", Rx_intfixed)

0     109.623016
1     486.792453
2    1047.024952
3    5077.186964
dtype: float64 
 0     103.323016
1     457.392453
2    1011.024952
3    5041.186964
dtype: float64


In [70]:
Rx_interr = np.sqrt(
    (df_inter_conn["max_voltage"] * 5e-3 / (df_inter_conn["I"] / 1000)) ** 2
    + (
        (df_inter_conn["max_current"] / 1000) * 5e-3
        * (df_inter_conn["U"] / df_inter_conn["I"] ** 2) * 1000
    ) ** 2
) / np.sqrt(3)

print(df_inter_conn["max_voltage"] * 5e-3, "\n", (df_inter_conn["max_current"] / 1000) * 5e-3)

print(df_inter_conn["U"] / df_inter_conn["I"] ** 2)
print(1 / (df_inter_conn["I"] / 1000))


0    0.006
1    0.006
2    0.006
3    0.015
Name: max_voltage, dtype: float64 
 0    0.000075
1    0.000015
2    0.000008
3    0.000008
Name: max_current, dtype: float64
0    0.010875
1    0.229619
2    1.004822
3    8.708725
dtype: float64
0      99.206349
1     471.698113
2     959.692898
3    1715.265866
Name: I, dtype: float64


In [71]:

print(Rx_interr)

0     0.343661
1     1.634011
2     3.324477
3    14.854686
dtype: float64


In [72]:
df_ext_conn = pd.DataFrame({
    "Rx0": [0, 500.0, 0, 5000.0],
    "internal_RV": [1199.9, 1199.9, 1199.9, 3000.0],
    "U": [1.126, 1.038, 0.802, 2.6800],
    "internal_RA": [0, 29.4, 0, 36.0],
    "max_voltage": [1.2, 1.2, 1.2, 3],
    "max_current": [15, 3, 1.5, 1.5],
    "I": [12.10, 2.926, 1.468, 1.423],
})

Rx_ext = df_ext_conn["U"] / df_ext_conn["I"] * 1000
Rx_extfixed = df_ext_conn["U"] / (
    df_ext_conn["I"] / 1000 - df_ext_conn["U"] / df_ext_conn["internal_RV"]
)

print(Rx_ext, "\n", Rx_extfixed)

0      93.057851
1     354.750513
2     546.321526
3    1883.345046
dtype: float64 
 0     100.881698
1     503.656627
2    1002.987743
3    5059.786029
dtype: float64


In [73]:
used_int_RVerr = pd.Series([int_RV_err[0]] * 3 + [int_RV_err[1]])

Rx_exterr = np.sqrt(
    (
        df_ext_conn["max_voltage"] * 5e-3 
        * df_ext_conn["internal_RV"] ** 2 * df_ext_conn["I"] / 1000
        / ((df_ext_conn["I"] / 1000 * df_ext_conn["internal_RV"] - df_ext_conn["U"])) ** 2
    ) ** 2
    + (
        (df_ext_conn["max_current"] / 1000) * 5e-3
        * df_ext_conn["U"]
        / ((df_ext_conn["I"] / 1000) - df_ext_conn["U"] / df_ext_conn["internal_RV"]) ** 2
    ) ** 2
    + (
        used_int_RVerr * (
            df_ext_conn["U"] ** 2
            / (df_ext_conn["I"] / 1000 * df_ext_conn["internal_RV"] - df_ext_conn["U"])
            ** 2
        )
    ) ** 2
) / np.sqrt(3)

print(df_ext_conn["max_current"] / 1000 * 5e-3, "\n", df_ext_conn["max_voltage"] * 5e-3, "\n", used_int_RVerr)

print(df_ext_conn["U"] / ((df_ext_conn["I"] / 1000) - df_ext_conn["U"] / df_ext_conn["internal_RV"]) ** 2)
print(df_ext_conn["internal_RV"] ** 2 * df_ext_conn["I"] / 1000 / ((df_ext_conn["I"] / 1000 * df_ext_conn["internal_RV"] - df_ext_conn["U"])) ** 2)
print(df_ext_conn["U"] ** 2 / (df_ext_conn["I"] / 1000 * df_ext_conn["internal_RV"] - df_ext_conn["U"]) ** 2)


0    0.000075
1    0.000015
2    0.000008
3    0.000008
Name: max_current, dtype: float64 
 0    0.006
1    0.006
2    0.006
3    0.015
Name: max_voltage, dtype: float64 
 0    2.625
1    2.625
2    2.625
3    3.000
dtype: float64
0    9.038292e+03
1    2.443834e+05
2    1.254345e+06
3    9.552774e+06
dtype: float64
0      97.125520
1     688.888159
2    2295.982486
3    5072.237904
dtype: float64
0    0.007069
1    0.176189
2    0.698717
3    2.844604
dtype: float64


In [74]:

print(Rx_exterr)

0     0.516222
1     3.200836
2     9.689203
3    60.538299
dtype: float64


In [75]:
# --- 整理并输出最终实验结果 ---

print("--- 1. 替代法测电压表内阻 ---")
# 将误差限转换为标准不确定度
for i in range(len(int_RV)):
    # 根据不确定度确定真值的保留小数位数
    # 规则：不确定度首位非零数在个位及以上，真值取整；在十分位，真值保留一位小数；以此类推
    if int_RV_err[i] >= 1:
        val_format = "{:.0f}"
    else:
        # 计算不确定度需要保留的小数位数
        uncertainty_digits = -int(np.floor(np.log10(int_RV_err[i]))) + 1
        val_format = "{:." + str(uncertainty_digits) + "f}"

    val_str = val_format.format(int_RV[i])
    print(f"电压表内阻 Rv{i+1}: ({val_str} ± {int_RV_err[i]:.2f}) Ω")

u_int_RV = np.array(int_RV_err) / np.sqrt(3)
for i in range(len(int_RV)):
    # 根据不确定度确定真值的保留小数位数
    # 规则：不确定度首位非零数在个位及以上，真值取整；在十分位，真值保留一位小数；以此类推
    if u_int_RV[i] >= 1:
        val_format = "{:.0f}"
    else:
        # 计算不确定度需要保留的小数位数
        uncertainty_digits = -int(np.floor(np.log10(u_int_RV[i]))) + 1
        val_format = "{:." + str(uncertainty_digits) + "f}"

    val_str = val_format.format(int_RV[i])
    print(f"电压表内阻 Rv{i+1}: ({val_str} ± {u_int_RV[i]:.2f}) Ω")

print("\n" + "--- 2. 电流表内接法测电阻 ---")
for i in range(len(df_inter_conn)):
    rx_val = df_inter_conn['Rx0'][i]
    measured_val = Rx_intfixed[i]
    uncertainty = Rx_interr[i]
    
    if uncertainty >= 1:
        val_format = "{:.0f}"
    else:
        uncertainty_digits = -int(np.floor(np.log10(uncertainty))) + 1
        val_format = "{:." + str(uncertainty_digits) + "f}"
        
    val_str = val_format.format(measured_val)
    print(f"标称值 {rx_val:.0f} Ω: Rx = ({val_str} ± {uncertainty:.2f}) Ω")


print("\n" + "--- 3. 电流表外接法测电阻 ---")
for i in range(len(df_ext_conn)):
    rx_val = df_ext_conn['Rx0'][i]
    measured_val = Rx_extfixed[i]
    uncertainty = Rx_exterr[i]

    if uncertainty >= 1:
        val_format = "{:.0f}"
    else:
        uncertainty_digits = -int(np.floor(np.log10(uncertainty))) + 1
        val_format = "{:." + str(uncertainty_digits) + "f}"

    val_str = val_format.format(measured_val)
    print(f"标称值 {rx_val:.0f} Ω: Rx = ({val_str} ± {uncertainty:.2f}) Ω")


--- 1. 替代法测电压表内阻 ---
电压表内阻 Rv1: (1200 ± 2.62) Ω
电压表内阻 Rv2: (3000 ± 3.00) Ω
电压表内阻 Rv3: (6003 ± 6.08) Ω
电压表内阻 Rv1: (1200 ± 1.52) Ω
电压表内阻 Rv2: (3000 ± 1.73) Ω
电压表内阻 Rv3: (6003 ± 3.51) Ω

--- 2. 电流表内接法测电阻 ---
标称值 100 Ω: Rx = (103.32 ± 0.34) Ω
标称值 500 Ω: Rx = (457 ± 1.63) Ω
标称值 1000 Ω: Rx = (1011 ± 3.32) Ω
标称值 5000 Ω: Rx = (5041 ± 14.85) Ω

--- 3. 电流表外接法测电阻 ---
标称值 0 Ω: Rx = (100.88 ± 0.52) Ω
标称值 500 Ω: Rx = (504 ± 3.20) Ω
标称值 0 Ω: Rx = (1003 ± 9.69) Ω
标称值 5000 Ω: Rx = (5060 ± 60.54) Ω
